# Plot UPD Data
## Run the following cells to import necessary libraries and define the required methods for plotting your data

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline
from collections import defaultdict

In [ ]:
valid_states = ['mat_i_upd', 'mat_a_upd', 'pat_i_upd', 'pat_a_upd', 'bpi',] 
def states_per_region(df, chrom, sample, w=1000000 ):
    counts = defaultdict(list)
    chrom_df = df[(df.chrom == chrom) & (df.Sample == sample)]
    i = chrom_df.iloc[0].pos
    while i <= chrom_df.iloc[-1].pos - w:
        window = chrom_df[(chrom_df.pos >= i) & (chrom_df.pos <= i + w)]
        i += w
        for state in valid_states:
            if len(window) == 0:
                continue    
            elif state == 'mat_a_upd':
                f = len(window[(window.state == state) | (window.state == 'mat_i_upd')])/len(window)
            elif state == 'pat_a_upd':
                f = len(window[(window.state == state) | (window.state == 'pat_i_upd')])/len(window)
            else:
                f = len(window[window.state == state])/len(window)
            counts['State'].append(state)
            counts['Frac'].append(f)
            counts['Calls'].append(len(window))
            counts['Pos'].append((i-w)/1e6) 
    return pd.DataFrame(counts)

In [ ]:
def plot_upd(df, sample, chrom, w=1000000, fig_dimensions=(12, 6), marker=None):
    states = states_per_region(df, sample=sample, chrom=chrom, w=w)
    colors = sns.color_palette("Paired", len(valid_states))
    lbls = {'mat_i_upd': 'Maternal isodisomic UPD',
            'mat_a_upd': 'Maternal total UPD',
            'pat_i_upd': 'Paternal isodisomic UPD',
            'pat_a_upd': 'Paternal total UPD',
            'bpi'      :  'Bi-parental'
           }
    fig = plt.figure(figsize=fig_dimensions)
    fig.suptitle("{} {}".format(sample, chrom))
    grid = plt.GridSpec(2, 2, wspace=0.4, hspace=0.3)
    fig.add_subplot(grid[0, 1],) 
    plt.plot(states.Pos, states.Calls, color='red', marker=marker,
             label="Calls per {:g} bp".format(w))
    plt.title("Calls per Window")
    plt.ylabel("Calls")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    fig.add_subplot(grid[1, 1],) 
    for i in range(len(valid_states)):
        s = states[states.State == valid_states[i]]
        plt.plot(s.Pos, s.Frac, color=colors[i],
                 label=lbls[valid_states[i]],  marker=marker)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title("States")
    plt.xlabel("Pos (Mb)")
    plt.ylabel("Fraction")
    pivot = states.pivot("State", "Pos", "Calls")
    fig.add_subplot(grid[0, 0],)
    ax = sns.heatmap(pivot)
    ax.set_title("Calls per Window")
    ax.xaxis.set_visible(False)
    majorFormatter = FormatStrFormatter('%.2f')
    fig.add_subplot(grid[1, 0],)
    pivot = states.pivot("State", "Pos", "Frac")
    ax = sns.heatmap(pivot)
    ax.set_title("States")
    majorFormatter = FormatStrFormatter('%.2f')
    ax.xaxis.set_major_formatter(majorFormatter)

In [ ]:
def plot_sig_upd(sample_file, pos_file, w=1e6, min_frac=0.05,
                 min_vs_other_prob=1e-5, min_vs_self_prob=1e-5):
    pos_state = pd.read_table(pos_file, names=["chrom", "pos", "Sample", "state"])
    sample_upd = pd.read_table(sample_file)
    sigs = sample_upd[(sample_upd['Test'] != 'Homozygosity') & 
                      (sample_upd['vs_chrom'] < min_vs_other_prob) & 
                      (sample_upd['vs_self'] < min_vs_self_prob) & 
                      (sample_upd['Fraction'] > min_frac)]
    for s in sigs.Sample.unique():
        for c in sigs[sigs.Sample == s].Chrom.unique():
            plot_upd(pos_state, sample=s, chrom=c, w=w)

# The cells below will allow you to plot your data - customize them as needed

In [ ]:
sns.set_context("talk") #change to another seaborn context as required
sns.set_style('darkgrid') #if you prefer a different background choose 
                          #either 'whitegrid', 'white' or 'dark' instead 

## The cell below will plot sample data per chromosome where there is a significant fraction of UPD genotypes

In [ ]:
results_file = ... #specify the output from updiva.py here to find 
                   #samples and chromosomes to plot
coord_file = ... #specify --coordinate_table output from updiva.py
                 #here for reading UPD state data
plot_sig_upd(results_file, coord_file, w=1e6, min_frac=0.05,
             min_vs_other_prob=1e-5,  min_vs_self_prob=1e-5)

## Alternatively read your result tables and inspect manually

In [ ]:
results = pd.read_table("your_results_file_here.tsv") 
coords = pd.read_table("your_coords_file_here.tsv.gz",  
                           names=["chrom", "pos", "Sample", "state"])

## Plot a specific sample and chromosome

In [ ]:
plot_upd(coords, sample='your_sample', chrom='chr1')